## 简答题
1. 如果已经在完全相同的训练集上训练了5个不同的模型，并且它们都达到了95%的准确率，是否还有机会获得更好的效果？如果可以，该怎么做？ 如果不行，为什么？
2. 硬投票分类器和软投票分类器有什么区别？
3. 是否可以通过在多个服务器上并行来加速bagging集成的训练？pasting集成呢？提升集成呢？随机森林或堆叠集成呢？
4. 包外评估的好处是什么？
5. 是什么让极端随机树比一般随机森林更加随机？这部分增加的随机性有什么用？极端随机树比一般随机森林快还是慢？
6. 如果AdaBoost集成对训练数据欠拟合，应该调整哪些超参数？怎么调整？
7. 如果梯度提升集成对训练集过拟合，应该提升还是降低学习率

## 编程题
1. 加载MNIST数据集（**8_sklearn做分类** 里有介绍），将其分为一个训练集、一个验证集和一个测试集（例如使用40000个实例训练，10000个实例验证，最后20000个实例测试）。然后训练多个分类器，比如一个随机森林分类器、一个极端随机树分类器和一个SVM。接下来，尝试使用软投票法或者硬投票法将它们组合成一个集成，这个集成在验证集上的表现要胜过它们各自单独的表现。成功找到集成后，在测试集上测试。与单个的分类器相比，它的性能要好多少？


2. 运行上一题中的单个分类器，用验证集进行预测，然后用预测结果创建一个新的训练集：新训练集中的每个实例都是一个向量，这个向量包含所有分类器对于一个图像的一组预测，目标值是图像的类。恭喜，你成功训练了一个混合器，结合第一层的分类器，它们一起构成了一个堆叠集成。现在在测试集上评估这个集成。对于测试集中的每个图像，使用所有的分类器进行预测，然后将预测结果提供给混合器，得到集成的预测。与前面训练的投票分类器相比，这个集成的结果如何？现在再次尝试使用StackingClassifier。你得到了更好的性能吗？如果是这样，为什么？

## 简答题

In [42]:
# 1. 如果已经在完全相同的训练集上训练了5个不同的模型，并且它们都达到了95%的准确率，是否还有机会获得更好的效果？如果可以，该怎么做？ 如果不行，为什么？
# 答：有机会获得更好的效果。
# 原因： 即使每个模型都有95%的准确率，但它们很可能会犯不同的错误。一个模型误分类的样本，可能被另一个模型正确分类。集成学习（Ensemble Learning）的核心思想就是通过组合多个模型，利用这种“多样性”来抵消单个模型的错误，从而获得比任何单一模型都更高的准确率、鲁棒性和泛化能力。
# 具体做法：
# 投票法/平均法： 对于分类任务，可以创建一个硬投票或软投票分类器（见问题2），让这5个模型进行投票，少数服从多数。这通常能提升模型的稳定性和准确率。

# 堆叠法： 这是一种更高级的集成技术。可以训练一个** blender （混合器）模型或元学习器**，它以这5个模型的预测结果作为输入特征，来学习如何最好地组合它们，从而做出最终预测。这通常比简单的投票法更强大。

# 检查多样性： 确保这5个模型是多样的（使用不同算法，如SVM、决策树、KNN、神经网络等）。如果5个模型是同一算法且超参数几乎相同，则集成效果可能不佳。多样性是集成有效的关键。

In [43]:
# 2. 硬投票分类器和软投票分类器有什么区别？
# 答： 它们的区别在于汇总各个分类器预测结果的方式不同。
# 硬投票分类器：
# 机制： 每个分类器对预测样本投出一票（一个类别标签），集成模型选择得票最多的类别作为最终预测结果。
# 例子： 三个分类器对某个样本的预测分别是 [A, A, B]，则硬投票的最终预测是 A（2票对1票）。
# 特点： 只关心每个分类器的最终决策结果，不关心其确定性程度。

# 软投票分类器：
# 机制： 每个分类器输出其对于每个类别的预测概率。集成模型将所有分类器的预测概率平均（或加权平均），然后选择平均概率最高的类别作为最终预测。
# 例子： 两个分类器对类别A的概率分别是 [0.51, 0.49]，对类别B的概率是 [0.49, 0.51]。硬投票会平票（A和B各一票），结果不确定。而软投票会计算平均概率：A的平均概率 = (0.51+0.49)/2 = 0.5；B的平均概率 = (0.49+0.51)/2 = 0.5。虽然这里也是平局，但如果第一个分类器对A的置信度很高（例如0.9），而第二个对B的置信度很低（例如0.6），软投票就能更合理地选择A。
# 特点： 考虑了分类器预测的“置信度”，通常比硬投票表现更好，但要求集合中的每个分类器都能够估计可靠的类别概率（例如，具有 predict_proba() 方法）。

In [44]:
# 3. 是否可以通过在多个服务器上并行来加速bagging集成的训练？pasting集成呢？提升集成呢？随机森林或堆叠集成呢？
# 答：这取决于集成方法的内在结构。
# Bagging 和 Pasting：可以。
# 原因： Bagging（自助采样）和 Pasting（不重复采样）的核心思想是每个基学习器都是独立地在训练集的一个子集上训练的。这意味着每个基学习器的训练过程之间没有任何依赖关系，可以完美地进行并行化，从而显著加速训练。

# 随机森林：可以。
# 原因： 随机森林是Bagging集成的一个特例，其基学习器全是决策树。同样，每棵树的构建是独立的，所以也可以并行训练。

# Boosting（提升集成）：不可以直接并行。
# 原因： Boosting（如AdaBoost, Gradient Boosting）是顺序的。每个基学习器（如一棵树）的训练都依赖于它前面所有学习器的结果（它需要针对前一个模型产生的错误进行优化）。这种强依赖性决定了它无法像Bagging那样轻松地并行训练整个集成过程。不过，在构建单棵树时的一些步骤（如寻找最佳分裂点）可以并行化，但整体集成流程是串行的。

# 堆叠集成：部分可以。
# 原因： 堆叠集成的训练分两层：
# 第一层（基学习器）： 所有基学习器可以并行训练，因为它们之间没有依赖关系（就像Bagging一样）。
# 第二层（元学习器）： 必须在所有基学习器训练完成并生成预测之后才能开始训练。这是一个串行点。
# 因此，基学习器的训练可以并行加速，但元学习器的训练不能。

In [45]:
# 4. 包外评估的好处是什么？
# 答：包外评估为Bagging方法（如随机森林）提供了一个内置的、几乎免费的验证集，无需额外分离数据。

In [46]:
# 5. 是什么让极端随机树比一般随机森林更加随机？这部分增加的随机性有什么用？极端随机树比一般随机森林快还是慢？
# 增加的随机性： 随机森林在构建每棵树时，分裂节点是在一个随机子集的特征中选择最佳分裂点。而极端随机树不仅随机选择特征子集，它还随机选择分裂阈值（为每个候选特征随机生成几个分裂阈值），然后选择这些随机阈值中最好的一个作为分裂规则。它不再计算所有可能阈值下的“最佳”点，而是随机生成几个。
# 增加随机性的作用：
# 进一步降低方差： 通过引入更多的随机性，使得每棵树之间的相关性更低。集成模型的泛化误差会随着基学习器之间相关性的降低而减小。
# 减少过拟合： 更强的正则化效果，使模型对训练数据的噪声更不敏感。
# 速度会更快。

In [47]:
# 6. 如果AdaBoost集成对训练数据欠拟合，应该调整哪些超参数？怎么调整？
# 答：AdaBoost对训练数据欠拟合，说明模型过于简单，无法捕捉数据中的模式。
# 可以调整以下超参数：
# 1.增加基学习器的数量： 这是最直接的方法。增加 n_estimators 参数，让集成模型有更多的“专家”来纠正前序的错误，从而提高模型能力。
# 2.增加基学习器的复杂度： AdaBoost默认使用决策树桩（深度为1的树），这是非常弱的模型。如果数据本身关系复杂，树桩就太简单了。可以增加基学习器的 max_depth（例如从1改为2或3），或者减少限制树复杂度的参数（如 min_samples_split），让每个基学习器变得更强大。
# 3.提高学习率： 降低 learning_rate 参数。AdaBoost中的学习率是收缩系数，降低它意味着每个基学习器的贡献变小，需要更多的学习器来拟合模型，整个过程更谨慎，有时能更好地收敛到一个更复杂的模型。但通常优先调整前两个参数。

In [48]:
# 7. 如果梯度提升集成对训练集过拟合，应该提升还是降低学习率
# 答：应该降低学习率。在梯度提升中，学习率（通常记为 η 或 learning_rate）是一个收缩超参数。它控制每棵新树对最终预测结果的贡献（或影响）程度。

## 编程题

In [49]:
# 1. 加载MNIST数据集（**8_sklearn做分类** 里有介绍），将其分为一个训练集、一个验证集和一个测试集（例如使用40000个实例训练，10000个实例验证，最后20000个实例测试）。然后训练多个分类器，比如一个随机森林分类器、一个极端随机树分类器和一个SVM。接下来，尝试使用软投票法或者硬投票法将它们组合成一个集成，这个集成在验证集上的表现要胜过它们各自单独的表现。成功找到集成后，在测试集上测试。与单个的分类器相比，它的性能要好多少？

In [50]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [51]:
mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")

In [52]:
# 1划分训练集，验证集和测试集
X=mnist.data
y=mnist.target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=20000,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=10000,random_state=42)

In [53]:
print(len(X_train),len(X_val),len(X_test))

40000 10000 20000


In [54]:
# 2选择模型,构建训练管道
models={
    'rfc': RandomForestClassifier(random_state=42),
    'etc': ExtraTreeClassifier(random_state=42),
    'svc': Pipeline([('scaler', StandardScaler()), ('svc', SVC(random_state=42,probability=True))])}

In [ ]:
# 3 随机搜索超参数
from sklearn.utils import resample
X_train_small, y_train_small = resample(X_train, y_train, n_samples=1000,random_state=42,stratify=y_train)

param_distributions = {
    'rfc': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 30, 50],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4] },
    'etc': {
        'max_depth': [None, 10, 30, 50],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4] },
    'svc': {
        'svc__C': [0.1, 1, 10],
        'svc__gamma': ['scale','auto'],
        'svc__kernel': ['rbf','poly','sigmoid'] }}

best_models = {}
best_params = {}
val_accuracies = {}

for name, model in models.items():
    train_data = X_train_small if name == 'svc' else X_train
    train_labels = y_train_small if name == 'svc' else y_train

    random_search = RandomizedSearchCV(
        model, param_distributions[name],
        n_iter=10,
        cv=3,
        scoring='accuracy',
        random_state=42)

    random_search.fit(X_train, y_train)

    # 记录最佳模型和参数
    best_models[name] = random_search.best_estimator_
    best_params[name] = random_search.best_params_

    # 在验证集上评估
    y_val_pred = best_models[name].predict(X_val)
    val_accuracies[name] = accuracy_score(y_val, y_val_pred)

    print(f"{name} 最佳参数: {random_search.best_params_}")
    print(f"{name} 验证集准确率: {val_accuracies[name]:.4f}")



In [ ]:
# 4 用软投票法和硬投票法集成模型
voting_hard = VotingClassifier(
    estimators=[(name, best_models[name]) for name in best_models.keys()],
    voting='hard')

# 软投票集成
voting_soft = VotingClassifier(
    estimators=[(name, best_models[name]) for name in best_models.keys()],
    voting='soft')

# 训练集成模型
voting_hard.fit(X_train, y_train)
voting_soft.fit(X_train, y_train)

# 在验证集上评估集成模型
voting_hard_val_pred = voting_hard.predict(X_val)
voting_hard_val_acc = accuracy_score(y_val, voting_hard_val_pred)
voting_soft_val_pred = voting_soft.predict(X_val)
voting_soft_val_acc = accuracy_score(y_val, voting_soft_val_pred)

val_accuracies['vh_acc'] = voting_hard_val_acc
val_accuracies['vs_acc'] = voting_soft_val_acc

print(f"硬投票集成验证集准确率: {voting_hard_val_acc:.4f}")
print(f"软投票集成验证集准确率: {voting_soft_val_acc:.4f}")

In [ ]:
# 5 对比单一模型和集成后的模型在测试集上的表现
test_accuracies = {}

for name, model in best_models.items():
    y_test_pred = model.predict(X_test)
    test_accuracies[name] = accuracy_score(y_test, y_test_pred)
    print(f"{name} 测试集准确率: {test_accuracies[name]:.4f}")

# 评估集成模型
voting_hard_test_pred = voting_hard.predict(X_test)
voting_hard_test_acc = accuracy_score(y_test, voting_hard_test_pred)

voting_soft_test_pred = voting_soft.predict(X_test)
voting_soft_test_acc = accuracy_score(y_test, voting_soft_test_pred)

test_accuracies['vh'] = voting_hard_test_acc
test_accuracies['vs'] = voting_soft_test_acc

print(f"硬投票集成测试集准确率: {voting_hard_test_acc:.4f}")
print(f"软投票集成测试集准确率: {voting_soft_test_acc:.4f}")

In [ ]:
# 6 可视化对比单一模型和集成模型的准确率 -AI
# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体
plt.rcParams['axes.unicode_minus'] = False  # 正确显示负号

# 创建可视化图表
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 验证集准确率对比
models_names = list(val_accuracies.keys())
val_acc_values = [val_accuracies[name] for name in models_names]
bars = ax1.bar(models_names, val_acc_values)
ax1.set_title('验证集准确率对比')
ax1.set_ylabel('准确率')
ax1.set_ylim(0.9, 1.0)
# 在柱状图上添加数值标签
for bar, acc in zip(bars, val_acc_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{acc:.4f}', ha='center', va='bottom')

# 测试集准确率对比
test_acc_values = [test_accuracies[name] for name in models_names]
bars = ax2.bar(models_names, test_acc_values)
ax2.set_title('测试集准确率对比')
ax2.set_ylabel('准确率')
ax2.set_ylim(0.9, 1.0)
# 在柱状图上添加数值标签
for bar, acc in zip(bars, test_acc_values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{acc:.4f}', ha='center', va='bottom')

# 准确率提升对比（相对于最好的单一模型）
best_single_model_acc = max([test_accuracies[name] for name in ['随机森林', '极端随机树', '支持向量机']])
improvement = {}
for name in test_accuracies:
    if name in ['硬投票集成', '软投票集成']:
        improvement[name] = test_accuracies[name] - best_single_model_acc

names = list(improvement.keys())
values = list(improvement.values())
bars = ax4.bar(names, values)
ax4.set_title('集成模型相对于最佳单一模型的准确率提升')
ax4.set_ylabel('准确率提升')
# 在柱状图上添加数值标签
for bar, imp in zip(bars, values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.0001 if height >= 0 else height - 0.0005,
             f'{imp:.4f}', ha='center', va='bottom' if height >= 0 else 'top')

plt.tight_layout()
plt.show()

In [ ]:
# 打印总结报告
print("\n===== 总结报告 =====")
print(f"最佳单一模型: {max([(name, acc) for name, acc in test_accuracies.items() if name not in ['硬投票集成', '软投票集成']], key=lambda x: x[1])[0]}")
print(f"最佳模型 overall: {max(test_accuracies.items(), key=lambda x: x[1])[0]}")
print(f"硬投票集成相对于最佳单一模型的提升: {test_accuracies['硬投票集成'] - best_single_model_acc:.4f}")
print(f"软投票集成相对于最佳单一模型的提升: {test_accuracies['软投票集成'] - best_single_model_acc:.4f}")

In [ ]:
# 2. 运行上一题中的单个分类器，用验证集进行预测，然后用预测结果创建一个新的训练集：新训练集中的每个实例都是一个向量，这个向量包含所有分类器对于一个图像的一组预测，目标值是图像的类。恭喜，你成功训练了一个混合器，结合第一层的分类器，它们一起构成了一个堆叠集成。现在在测试集上评估这个集成。对于测试集中的每个图像，使用所有的分类器进行预测，然后将预测结果提供给混合器，得到集成的预测。与前面训练的投票分类器相比，这个集成的结果如何？现在再次尝试使用StackingClassifier。你得到了更好的性能吗？如果是这样，为什么？

In [ ]:
# 初始化第一层分类器
rf_clf = RandomForestClassifier(random_state=42)
et_clf = ExtraTreeClassifier(random_state=42)
svm_clf = SVC(random_state=42)

# 训练分类器
rf_clf.fit(X_train, y_train)
et_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)

# 在验证集上进行预测
rf_pred = rf_clf.predict(X_val)
et_pred = et_clf.predict(X_val)
svm_pred = svm_clf.predict(X_val)

# 创建新的训练集（堆叠训练集）
X_train_stack = np.column_stack((rf_pred, et_pred, svm_pred))
y_train_stack = y_val

print(f"随机森林准确率: {accuracy_score(y_val, rf_pred):.4f}")
print(f"极端随机树准确率: {accuracy_score(y_val, et_pred):.4f}")
print(f"SVM准确率: {accuracy_score(y_val, svm_pred):.4f}")

In [ ]:
# 使用逻辑回归作为混合器
from sklearn.linear_model import LogisticRegression

blender = LogisticRegression(random_state=42)
blender.fit(X_train_stack, y_train_stack)

# 在测试集上评估整个集成
rf_test_pred = rf_clf.predict(X_test)
et_test_pred = et_clf.predict(X_test)
svm_test_pred = svm_clf.predict(X_test)

X_test_stack = np.column_stack((rf_test_pred, et_test_pred, svm_test_pred))
y_pred_stack = blender.predict(X_test_stack)

stacking_accuracy = accuracy_score(y_test, y_pred_stack)
print(f"\n堆叠集成在测试集上的准确率: {stacking_accuracy:.4f}")

In [ ]:
# 创建并训练投票分类器
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf), ('svm', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)
voting_accuracy = accuracy_score(y_test, y_pred_voting)

print(f"投票分类器在测试集上的准确率: {voting_accuracy:.4f}")

In [ ]:
from sklearn.ensemble import StackingClassifier

# 使用相同的基分类器
base_classifiers = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('et', ExtraTreeClassifier(random_state=42)),
    ('svc', SVC(random_state=42))]

# 创建StackingClassifier
stack_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=LogisticRegression(random_state=42),
    cv=5
)

# 训练堆叠分类器
stack_clf.fit(X_train, y_train)
y_pred_skstack = stack_clf.predict(X_test)
skstack_accuracy = accuracy_score(y_test, y_pred_skstack)

print(f"Scikit-learn StackingClassifier在测试集上的准确率: {skstack_accuracy:.4f}")

In [ ]:
# 可视化比较结果 -AI
methods = ['随机森林', '极端随机树', 'SVM', '投票分类器', '手动堆叠', 'Scikit-learn堆叠']
accuracies = [
    accuracy_score(y_test, rf_clf.predict(X_test)),
    accuracy_score(y_test, et_clf.predict(X_test)),
    accuracy_score(y_test, svm_clf.predict(X_test)),
    voting_accuracy,
    stacking_accuracy,
    skstack_accuracy
]

plt.figure(figsize=(12, 6))
bars = plt.bar(methods, accuracies, color=['blue', 'blue', 'blue', 'green', 'red', 'orange'])
plt.ylabel('准确率')
plt.title('不同分类器性能比较')
plt.ylim(0.9, 1.0)

# 在柱状图上添加数值标签
for bar, accuracy in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{accuracy:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 分析结果
for i in range(3):
    print(f"   {methods[i]}: {accuracies[i]:.4f}")

print(f"2. 投票分类器比最好的基分类器提高了: {voting_accuracy - max(accuracies[:3]):.4f}")
print(f"3. 手动堆叠集成比投票分类器提高了: {stacking_accuracy - voting_accuracy:.4f}")
print(f"4. Scikit-learn堆叠比手动堆叠提高了: {skstack_accuracy - stacking_accuracy:.4f}")
